# Unpermuted Pixel MNIST Demo 
Light weighted demo of our DilatedRNN on Pixel MNist with permutation.

In [1]:
import sys
sys.path.append("./models")
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from classification_models import drnn_classification

In [2]:
# configurations
data_dir = "./MNIST_data"
n_steps = 28*28
input_dims = 1
n_classes = 10 

# model config
cell_type = "LSTM"
assert(cell_type in ["RNN", "LSTM", "GRU"])
hidden_structs = [20] * 9
dilations = [1, 2, 4, 8, 16, 32, 64, 128, 256]
assert(len(hidden_structs) == len(dilations))

# learning config
batch_size = 128
learning_rate = 1.0e-3
training_iters = batch_size * 30000
testing_step = 5000
display_step = 100

In [3]:
mnist = input_data.read_data_sets(data_dir, one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# build computation graph
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, n_steps, input_dims])
y = tf.placeholder(tf.float32, [None, n_classes])    
global_step = tf.Variable(0, name='global_step', trainable=False)

# build prediction graph
print ("==> Building a dRNN with %s cells" %cell_type)
pred = drnn_classification(x, hidden_structs, dilations, n_steps, n_classes, input_dims, cell_type)

# build loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate, 0.9).minimize(cost, global_step=global_step)

# evaluation model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

==> Building a dRNN with LSTM cells
Building layer: multi_dRNN_dilation_1, input length: 784, dilation rate: 1, input dim: 1.
=====> Input length for sub-RNN: 784
Building layer: multi_dRNN_dilation_2, input length: 784, dilation rate: 2, input dim: 20.
=====> Input length for sub-RNN: 392
Building layer: multi_dRNN_dilation_4, input length: 784, dilation rate: 4, input dim: 20.
=====> Input length for sub-RNN: 196
Building layer: multi_dRNN_dilation_8, input length: 784, dilation rate: 8, input dim: 20.
=====> Input length for sub-RNN: 98
Building layer: multi_dRNN_dilation_16, input length: 784, dilation rate: 16, input dim: 20.
=====> Input length for sub-RNN: 49
Building layer: multi_dRNN_dilation_32, input length: 784, dilation rate: 32, input dim: 20.
=====> 16 time points need to be padded. 
=====> Input length for sub-RNN: 25
Building layer: multi_dRNN_dilation_64, input length: 784, dilation rate: 64, input dim: 20.
=====> 48 time points need to be padded. 
=====> Input length

In [5]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
step = 0
train_results = []
validation_results = []
test_results = []

while step * batch_size < training_iters:
    batch_x, batch_y = mnist.train.next_batch(batch_size)    
    batch_x = batch_x.reshape([batch_size, n_steps, input_dims])

    feed_dict = {
        x : batch_x,
        y : batch_y
    }
    cost_, accuracy_, step_,  _ = sess.run([cost, accuracy, global_step, optimizer], feed_dict=feed_dict)    
    train_results.append((step_, cost_, accuracy_))    

    if (step + 1) % display_step == 0:
        print ("Iter " + str(step + 1) + ", Minibatch Loss: " + "{:.6f}".format(cost_) \
        + ", Training Accuracy: " + "{:.6f}".format(accuracy_))
             
    if (step + 1) % testing_step == 0:
        
        # validation performance
        batch_x = mnist.validation.images
        batch_y = mnist.validation.labels

        # unpermute the data    
        batch_x = batch_x.reshape([-1, n_steps, input_dims])
        feed_dict = {
            x : batch_x,
            y : batch_y
        }
        cost_, accuracy__, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
        validation_results.append((step_, cost_, accuracy__))
        
        # test performance
        batch_x = mnist.test.images
        batch_y = mnist.test.labels       
        batch_x = batch_x.reshape([-1, n_steps, input_dims])
        feed_dict = {
            x : batch_x,
            y : batch_y
        }
        cost_, accuracy_, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
        test_results.append((step_, cost_, accuracy_))        
        print ("========> Validation Accuarcy: " + "{:.6f}".format(accuracy__) \
        + ", Testing Accuarcy: " + "{:.6f}".format(accuracy_)) 
    step += 1

Iter 100, Minibatch Loss: 2.747471, Training Accuracy: 0.078125
Iter 200, Minibatch Loss: 2.301859, Training Accuracy: 0.109375
Iter 300, Minibatch Loss: 2.153563, Training Accuracy: 0.203125
Iter 400, Minibatch Loss: 1.727665, Training Accuracy: 0.367188
Iter 500, Minibatch Loss: 1.527061, Training Accuracy: 0.367188
Iter 600, Minibatch Loss: 1.462312, Training Accuracy: 0.453125
Iter 700, Minibatch Loss: 1.258504, Training Accuracy: 0.593750
Iter 800, Minibatch Loss: 1.378709, Training Accuracy: 0.531250
Iter 900, Minibatch Loss: 1.230721, Training Accuracy: 0.609375
Iter 1000, Minibatch Loss: 1.170994, Training Accuracy: 0.593750
Iter 1100, Minibatch Loss: 0.954037, Training Accuracy: 0.710938
Iter 1200, Minibatch Loss: 0.882938, Training Accuracy: 0.742188
Iter 1300, Minibatch Loss: 0.941144, Training Accuracy: 0.710938
Iter 1400, Minibatch Loss: 0.860346, Training Accuracy: 0.703125
Iter 1500, Minibatch Loss: 0.673839, Training Accuracy: 0.765625
Iter 1600, Minibatch Loss: 0.76275

Iter 12500, Minibatch Loss: 0.030145, Training Accuracy: 0.992188
Iter 12600, Minibatch Loss: 0.182634, Training Accuracy: 0.945312
Iter 12700, Minibatch Loss: 0.010769, Training Accuracy: 0.992188
Iter 12800, Minibatch Loss: 0.240914, Training Accuracy: 0.953125
Iter 12900, Minibatch Loss: 0.002995, Training Accuracy: 1.000000
Iter 13000, Minibatch Loss: 0.031563, Training Accuracy: 0.992188
Iter 13100, Minibatch Loss: 0.103042, Training Accuracy: 0.968750
Iter 13200, Minibatch Loss: 0.039135, Training Accuracy: 0.984375
Iter 13300, Minibatch Loss: 0.003223, Training Accuracy: 1.000000
Iter 13400, Minibatch Loss: 0.007587, Training Accuracy: 1.000000
Iter 13500, Minibatch Loss: 0.067823, Training Accuracy: 0.976562
Iter 13600, Minibatch Loss: 0.009642, Training Accuracy: 1.000000
Iter 13700, Minibatch Loss: 0.027099, Training Accuracy: 0.992188
Iter 13800, Minibatch Loss: 0.015181, Training Accuracy: 1.000000
Iter 13900, Minibatch Loss: 0.055021, Training Accuracy: 0.984375
Iter 14000